In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torchvision as tv

seed = 3
random.seed(seed)
torch.manual_seed(seed)

In [10]:
dataroot = "data"
workers = 2
batch_size = 128
image_size = 64
num_epochs = 5
lr = 0.0002
beta = 0.5
ngpu = 0

In [11]:
tfs = tv.transforms.Compose([
    tv.transforms.Resize(image_size),
    tv.transforms.CenterCrop(image_size),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = tv.datasets.ImageFolder(root=dataroot, transform=tfs)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

device = "cpu"
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [12]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    if classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

In [13]:
nc = 3   # Number of channels in image
nz = 100 # Latent vector size
ngf = 64 # Generator feature map sizes
ndf = 64 # Discriminator feature map sizes

class Up(nn.Module):
    def __init__(self, inc, outc, k, s, p):
        super(Up, self).__init__()
        self.m = nn.Sequential(
            nn.ConvTranspose2d(inc, outc, k, s, p, bias=False),
            nn.BatchNorm2d(outc),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.m(x)
    
    
class Down(nn.Module):
    def __init__(self, inc, outc, k, s, p):
        super(Down, self).__init__()
        self.m = nn.Sequential(
            nn.Conv2d(inc, outc, k, s, p, bias=False),
            nn.BatchNorm2d(outc),
            nn.LeakyReLU(0.2, inplace=True)
        )
    def forward(self, x):
        return self.m(x)
    

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.m = nn.Sequential(
            Up(   nz, 8*ngf, 4, 1, 0), # Out: 4x4
            Up(8*ngf, 4*ngf, 4, 2, 1), # Out: 8x8
            Up(4*ngf, 2*ngf, 4, 2, 1), # Out: 16x16
            Up(2*ngf,   ngf, 4, 2, 1), # Out: 32x32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False), # Out: 64x64
            nn.Tanh()
        )
    def forward(self, x):
        return self.m(x)
    

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.m = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1), # Out: 32x32
            nn.LeakyReLU(0.2, inplace=True),
            Down(  ndf, 2*ndf, 4, 2, 1), # Out: 16x16
            Down(2*ndf, 4*ndf, 4, 2, 1), # Out: 8x8
            Down(4*ndf, 8*ndf, 4, 2, 1), # Out: 4x4
            nn.Conv2d(8*ndf, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.m(x)

In [14]:
netG = Generator(ngpu).to(device)
netD = Discriminator(ngpu).to(device)

if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))
    netD = nn.DataParallel(netD, list(range(ngpu)))
    
netG.apply(weights_init)
netD.apply(weights_init)

# l(x, y) = y*log(x) + (1-y)*log(1-x)
# if (y==1): l(x, y) = log(x)   = 0 if (x==1) else negative
# if (y==0): l(x, y) = log(1-x) = 0 if (x==0) else negative

criterion = nn.BCELoss()
fixed_noise = torch.randn(64, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

optimizerD = torch.optim.Adam(
    netD.parameters(), lr=lr, betas=(beta, 0.999))
optimizerG = torch.optim.Adam(
    netG.parameters(), lr=lr, betas=(beta, 0.999))

In [15]:
img_list = []
G_losses = []
D_losses = []
iters = 0

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        
        # Train D
        netD.zero_grad()
        
        # Get batch of real images, labeled as real
        real_data = data[0].to(device)
        b_size = real_data.size(0)
        label = torch.full((b_size,), real_label, device=device)
        
        # Run discriminator and accumulate
        output = netD(real_data).view(-1)
        errD_real = criterion(output, label) # large if output != 1
        errD_real.backward()
        D_x = output.mean().item()
        
        # Generate fake images from generator
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake_data = netG(noise)
        
        # Run discriminator and accumulate
        label.fill_(fake_label)
        output = netD(fake_data.detach()).view(-1)
        errD_fake = criterion(output, label) # large if output != 0
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        
        # Step the disciminator
        errD = errD_real + errD_fake
        optimizerD.step()
        
        # Compute loss on flipped labels
        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake_data).view(-1) # D's been updated, re-run
        errG = criterion(output, label) # small if == 1, i.e. fooled discriminator
        errG.backward()
        
        # Step the generator
        D_G_z2 = output.mean().item()
        optimizerG.step()
        
        # Print updates
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
            
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        if ((iters % 500 == 0) or
            (epoch == num_epochs-1 and i == len(dataloader-1))):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(tv.utils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1

[0/5][0/1583]	Loss_D: 1.3827	Loss_G: 0.7873	D(x): 0.5025	D(G(z)): 0.5007 / 0.4551
[0/5][50/1583]	Loss_D: 0.1081	Loss_G: 2.9221	D(x): 0.9518	D(G(z)): 0.0568 / 0.0540
[0/5][100/1583]	Loss_D: 0.0300	Loss_G: 4.1089	D(x): 0.9874	D(G(z)): 0.0171 / 0.0166
[0/5][150/1583]	Loss_D: 0.4407	Loss_G: 3.2801	D(x): 0.8438	D(G(z)): 0.2047 / 0.0405


Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 